# Retail Sales Analysis & Forecasting (5-Year Period)

## Project Overview
This project analyzes five years of retail sales data to uncover sales trends,
identify best-selling products and categories, and forecast future demand using
time-series analysis techniques.

The dataset contains intentional data quality issues such as missing values,
inconsistent product naming, and duplicated records to simulate real-wor data.
issues.


## Environment Setup
In this section, we import the required Python libraries used for data manipulation,
visualization, and time-series modeling.

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("data/raw/dirty_retail_sales_5years_9000plus.csv")
df.head()

## Data Overview
The dataset contains transactional retail sales records spanning five years.
Initial inspection reveals missing values, duplicated records, and inconsistent
text formatting across product and category columns.

In [ ]:
# First 5 rows
df.head()

# Last 5 rows
df.tail()


In [ ]:
#checking the size of the dataset
df.shape

In [ ]:
#checking the column type and name
df.info()

## Data Quality Assessment
This section examines missing values, duplicated records, and potential anomalies
that could affect analysis accuracy.

In [ ]:
#Quick Summary Statistics
df.describe()

In [ ]:
#Look at unique values in categorical columns
df['store'].unique()
df['product'].unique()
df['category'].unique()
df['payment_method'].unique()

In [ ]:
#checking for missing values
df.isna().sum()

In [ ]:
#checking for duplicates
df.duplicated().sum()

In [ ]:
#exploring numeric anomalies
# Negative sales
df[df['units_sold'] < 0]

# Strange prices (optional: negative or zero)
df[df['unit_price'] <= 0]

In [ ]:
#Explore categorical inconsistencies
df = df.drop_duplicates()

In [ ]:
# Option: drop rows with missing date (needed for time series)
df = df.dropna(subset=['date'])

In [ ]:
#Handle missing categorical data
df['discount_applied'] = df['discount_applied'].fillna(0)   # No discount if missing
df['payment_method'] = df['payment_method'].fillna('unknown')

In [ ]:
# Standardize the text column, convert all text to lowercase and remove extra spaces 
df['store'] = df['store'].str.lower().str.strip()
df['category'] = df['category'].str.lower().str.strip()
df['payment_method'] = df['payment_method'].str.lower().str.strip()

## Data Cleaning Summary
Data cleaning was performed to standardize product and category names, handle
missing values, and remove duplicate records. These steps ensure accurate
aggregation and reliable trend analysis.


In [ ]:
# check  for anomalies in numeric column Negative units_sold
returns = df[df['units_sold'] < 0]
returns.head()

In [ ]:
#add total sales column 
df['total_sales'] = df['units_sold'] * df['unit_price']

In [ ]:
#overall sales trends
# Total revenue
total_revenue = df['total_sales'].sum()
print(f"Total Revenue: {total_revenue}")

# Total units sold
total_units = df['units_sold'].sum()
print(f"Total Units Sold: {total_units}")

In [ ]:
#top selling products 
top_products = df.groupby('product')['total_sales'].sum().sort_values(ascending=False).head(10)
top_products

In [ ]:
#striping spaces and turning to lowecase
df['product'] = df['product'].str.lower().str.strip()

In [ ]:
# Standardize product names
df['product'] = df['product'].replace({
    'rice-5 kg': 'rice 5kg',
    'sugar 1 kg': 'sugar 1kg',
    'cooking oil 1 liter': 'cooking oil 1l',
    'detergent-2 kg': 'detergent 2kg'
})

In [ ]:
#top selling products 
top_products = df.groupby('product')['total_sales'].sum().sort_values(ascending=False).head(10)
top_products

In [ ]:
# Remove hyphens from product names
df['product'] = df['product'].str.replace('-', ' ', regex=False)

In [ ]:
df['product'].value_counts().head(10)

In [ ]:
# Standardize semantic duplicates
df['product'] = df['product'].replace({
    'tooth paste': 'toothpaste'})

In [ ]:
df.groupby('product')['total_sales'].sum().sort_values(ascending=False).head(10)

In [ ]:
# Revenue by category
category_sales = (
    df.groupby('category')['total_sales']
      .sum()
      .sort_values(ascending=False)
)
category_sales

In [ ]:
#fix the anomalies change to lower case
df['category'] = df['category'].str.lower().str.strip()

In [ ]:
#remove the space
df['category'] = df['category'].str.replace(' ', '', regex=False)

In [ ]:
#change the plural
df['category'] = df['category'].replace({
    'groceries': 'grocery'
})

In [ ]:
#recalculate the category sales
category_sales = (
    df.groupby('category')['total_sales']
      .sum()
      .sort_values(ascending=False)
)

category_sales


In [ ]:
# Revenue by store
store_sales = (
    df.groupby('store')['total_sales']
      .sum()
      .sort_values(ascending=False)
)

store_sales

## Exploratory Data Analysis
Analysis shows that a small number of products contribute a large share of total
revenue. Sales are dominated by grocery and household categories, highlighting
core demand drivers in the retail store.


In [ ]:
#validateing the date range
df['date'].min(), df['date'].max()

In [ ]:
# Convert date column to datetime
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df.dtypes

In [ ]:
monthly_sales = (
    df.set_index('date')
      .resample('M')['total_sales']
      .sum()
)

monthly_sales.head()

In [ ]:
monthly_sales.plot(
    figsize=(12,6),
    title='Monthly Sales Trend'
)

In [ ]:
# Train-test split for time series
train_size = int(len(monthly_sales) * 0.8)

train = monthly_sales.iloc[:train_size]
test = monthly_sales.iloc[train_size:]

train.shape, test.shape

In [ ]:
# Step 6.2: Naive forecast
naive_forecast = test.copy()
naive_forecast[:] = train.iloc[-1]

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))
plt.plot(train.index, train, label='Train')
plt.plot(test.index, test, label='Actual')
plt.plot(test.index, naive_forecast, label='Naive Forecast')
plt.legend()
plt.title("Naive Forecast vs Actual Sales")
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error

mae_naive = mean_absolute_error(test, naive_forecast)
mae_naive

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
# Fit Holt-Winters model (trend + seasonality)
hw_model = ExponentialSmoothing(
    train,
    trend='add',
    seasonal='add',
    seasonal_periods=12
).fit()

In [ ]:
hw_model = ExponentialSmoothing(
    train,
    trend='add',
    seasonal='add',
    seasonal_periods=12
).fit(smoothing_level=0.5, smoothing_slope=0.2, smoothing_seasonal=0.3)

In [ ]:
hw_model = ExponentialSmoothing(
    train,
    trend='add',
    seasonal='add',
    seasonal_periods=12
).fit(
    smoothing_level=0.5, 
    smoothing_trend=0.2,       # use this instead of smoothing_slope
    smoothing_seasonal=0.3
)

In [ ]:
from sklearn.metrics import mean_absolute_error

mae_hw = mean_absolute_error(test, hw_forecast)
mae_hw

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

hw_model = ExponentialSmoothing(
    train,
    trend='add',
    seasonal='add',
    seasonal_periods=12
).fit(
    smoothing_level=0.5, 
    smoothing_trend=0.2,
    smoothing_seasonal=0.3
)

In [ ]:
hw_forecast = hw_model.forecast(len(test))

In [ ]:
from sklearn.metrics import mean_absolute_error

mae_hw = mean_absolute_error(test, hw_forecast)
mae_hw

mape_hw = (abs(test - hw_forecast) / test).mean() * 100
mape_hw

In [ ]:
train_size = int(len(monthly_sales) * 0.8)
train = monthly_sales.iloc[:train_size]
test = monthly_sales.iloc[train_size:]

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

hw_model = ExponentialSmoothing(
    train,
    trend='add',
    seasonal='add',
    seasonal_periods=12
).fit(
    smoothing_level=0.5,
    smoothing_trend=0.2,
    smoothing_seasonal=0.3
)

In [ ]:
hw_forecast = hw_model.forecast(len(test))

In [ ]:
from sklearn.metrics import mean_absolute_error

# MAE
mae_hw = mean_absolute_error(test, hw_forecast)
print("Mean Absolute Error (MAE):", mae_hw)

# MAPE
mape_hw = (abs(test - hw_forecast) / test).mean() * 100
print("Mean Absolute Percentage Error (MAPE):", mape_hw)

## Time-Series Analysis & Forecasting
Monthly sales exhibit strong seasonal patterns with recurring peaks across years.
The Holt-Winters model captured trend and seasonality effectively, achieving a
MAPE of 9.48%, indicating good forecasting performance.


## Conclusion
This analysis demonstrates the importance of data cleaning in retail analytics.
Accurate forecasting can support inventory planning and demand management.
Future work could incorporate external factors such as promotions or holidays.